In [41]:
import pandas as pd
import joblib

from functools import reduce

# Charger le modèle entraîné
xgb = joblib.load("xgb_time_based.pkl")
print("✅ Model loaded")

# Charger le train_time_based pour récupérer les catégories de last_level
train_df = pd.read_parquet("train_time_based.parquet")
print("train_df shape:", train_df.shape)

# Récupérer les catégories de last_level (ordre stable)
last_level_cats = train_df["last_level"].astype("category").cat.categories
print("last_level categories:", list(last_level_cats))

✅ Model loaded
train_df shape: (15276, 21)
last_level categories: [0, 1]


In [42]:
# Reconstruire X_train_ref pour récupérer l'ordre et les features exactes
# ⚠️ Le modèle entraîné dans 02_modelisation2 n'utilise PAS last_level
X_train_ref = train_df.drop(columns=["userId", "target", "last_level"]).copy()

cols_order = X_train_ref.columns
print("Colonnes de référence (train, sans last_level) :", list(cols_order))


Colonnes de référence (train, sans last_level) : ['n_events', 'n_active_days', 'n_unique_pages', 'n_sessions', 'events_per_day', 'recency_days', 'account_age_days', 'n_help', 'n_upgrade', 'n_submit_downgrade', 'n_settings', 'n_about', 'n_error', 'n_songs_played', 'n_unique_songs', 'avg_song_length', 'most_active_dow', 'most_active_hour']


In [ ]:
test_path = "/Users/alexandre/Desktop/X/Python for Data Science/Projet Final Churn/test.parquet"
test = pd.read_parquet(test_path)

example = pd.read_csv("example_submission.csv")

print("test shape:", test.shape)
display(test.head())
display(example.head())


test shape: (4393179, 19)


,status,gender,firstName,level,lastName,userId,ts,auth,page,sessionId,location,itemInSession,userAgent,method,length,song,artist,time,registration
7,200,M,Jonathan,free,Martin,1465194,1538352006000,Logged In,NextSong,22483,"New York-Newark-Jersey City, NY-NJ-PA",29,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,250.82730,Mockingbird,Eminem,2018-10-01 00:00:06,2018-09-27 17:29:36
54,200,M,Jonathan,free,Martin,1465194,1538352028000,Logged In,Roll Advert,22483,"New York-Newark-Jersey City, NY-NJ-PA",30,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",GET,NaN,None,None,2018-10-01 00:00:28,2018-09-27 17:29:36
477,200,M,Jonathan,free,Martin,1465194,1538352256000,Logged In,NextSong,22483,"New York-Newark-Jersey City, NY-NJ-PA",31,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,355.78730,Thank You (Precious Memories Album Version),Ray Boltz,2018-10-01 00:04:16,2018-09-27 17:29:36
1170,200,M,Jonathan,free,Martin,1465194,1538352611000,Logged In,NextSong,22483,"New York-Newark-Jersey City, NY-NJ-PA",32,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,191.68608,Mathletics,Foals,2018-10-01 00:10:11,2018-09-27 17:29:36
1552,200,M,Jonathan,free,Martin,1465194,1538352802000,Logged In,NextSong,22483,"New York-Newark-Jersey City, NY-NJ-PA",33,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,275.25179,Proceed,The Roots,2018-10-01 00:13:22,2018-09-27 17:29:36


,id,target
0,1128274,0
1,1782451,1
2,1611542,1
3,1241663,0
4,1653104,1


In [44]:
test["ts"] = pd.to_datetime(test["ts"], unit="ms")

T_test = test["ts"].max()
print("Max ts in test:", T_test)


Max ts in test: 2018-11-20 00:00:00


In [45]:
test["date"] = test["ts"].dt.date

# 1) total events
f_events_test = (
    test.groupby("userId")
        .size()
        .reset_index(name="n_events")
)

# 2) active days
f_days_test = (
    test.groupby("userId")["date"]
        .nunique()
        .reset_index(name="n_active_days")
)

# 3) distinct pages
f_pages_test = (
    test.groupby("userId")["page"]
        .nunique()
        .reset_index(name="n_unique_pages")
)

# 4) sessions
f_sessions_test = (
    test.groupby("userId")["sessionId"]
        .nunique()
        .reset_index(name="n_sessions")
)

feature_dfs_test = [f_events_test, f_days_test, f_pages_test, f_sessions_test]

user_features_test = reduce(
    lambda left, right: pd.merge(left, right, on="userId", how="left"),
    feature_dfs_test
)

display(user_features_test.head())
print("user_features_test shape:", user_features_test.shape)


,userId,n_events,n_active_days,n_unique_pages,n_sessions
0,1000655,346,11,11,10
1,1000963,2539,26,17,21
2,1001129,668,10,15,8
3,1001963,718,16,14,16
4,1002283,3837,25,17,24


user_features_test shape: (2904, 5)


In [46]:
### 5) FEATURES COMPORTEMENTALES (comme dans le train)

# 5.1) Comptage des pages critiques
important_pages = ["Help", "Upgrade", "Submit Downgrade", "Settings", "About", "Error"]

page_features_test = pd.DataFrame({"userId": test["userId"].unique()})

for page in important_pages:
    page_name = f"n_{page.lower().replace(' ', '_')}"
    page_counts = test[test["page"] == page].groupby("userId").size().reset_index(name=page_name)
    page_features_test = page_features_test.merge(page_counts, on="userId", how="left")

page_features_test = page_features_test.fillna(0)

# 5.2) Features d'engagement musical
song_data_test = test[test["page"] == "NextSong"].copy()
if len(song_data_test) > 0:
    song_engagement_test = (
        song_data_test.groupby("userId")
        .agg({
            "length": ["count", "sum"],
            "song": "nunique"
        })
        .reset_index()
    )
    song_engagement_test.columns = ["userId", "n_songs_played", "total_listening_time", "n_unique_songs"]
    song_engagement_test["avg_song_length"] = song_engagement_test["total_listening_time"] / song_engagement_test["n_songs_played"]
    song_engagement_test = song_engagement_test[["userId", "n_songs_played", "n_unique_songs", "avg_song_length"]]
else:
    song_engagement_test = pd.DataFrame(columns=["userId", "n_songs_played", "n_unique_songs", "avg_song_length"])

# 5.3) Features temporelles
test["day_of_week"] = test["ts"].dt.dayofweek
test["hour"] = test["ts"].dt.hour

time_features_test = (
    test.groupby("userId")
    .agg({
        "day_of_week": lambda x: x.mode()[0] if len(x.mode()) > 0 else 0,
        "hour": lambda x: x.mode()[0] if len(x.mode()) > 0 else 12,
    })
    .reset_index()
    .rename(columns={"day_of_week": "most_active_dow", "hour": "most_active_hour"})
)

print("✅ Features comportementales calculées")



✅ Features comportementales calculées


In [47]:
### 1) RECENCY — days since last activity (par rapport à T_test)

last_ts_test = (
    test.groupby("userId")["ts"]
        .max()
        .reset_index(name="last_ts")
)

last_ts_test["recency_days"] = (T_test - last_ts_test["last_ts"]).dt.days
last_ts_test = last_ts_test[["userId", "recency_days"]]


### 2) ACCOUNT AGE — days since registration (par rapport à T_test)

test["registration"] = pd.to_datetime(test["registration"], unit="ms")

reg_age_test = (
    test.groupby("userId")["registration"]
        .min()
        .reset_index(name="registration_dt")
)

reg_age_test["account_age_days"] = (T_test - reg_age_test["registration_dt"]).dt.days
reg_age_test = reg_age_test[["userId", "account_age_days"]]


### 3) LAST LEVEL — dernier statut connu (free/paid)

last_level_test = (
    test.groupby("userId")["level"]
        .last()
        .reset_index()
        .rename(columns={"level": "last_level"})
)


### 4) EVENTS PER DAY — intensité d’activité

user_features_test["events_per_day"] = (
    user_features_test["n_events"] / user_features_test["n_active_days"]
)


In [48]:
feature_dfs_test_extended = [
    user_features_test,
    last_ts_test,
    reg_age_test,
    last_level_test,
    page_features_test
]

if len(song_engagement_test) > 0:
    feature_dfs_test_extended.append(song_engagement_test)

feature_dfs_test_extended.append(time_features_test)

test_features = reduce(
    lambda left, right: pd.merge(left, right, on="userId", how="left"),
    feature_dfs_test_extended
)

# Remplacer les NaN par 0 pour les nouvelles features
new_feature_cols = ["n_help", "n_upgrade", "n_submit_downgrade", "n_settings", "n_about", "n_error",
                    "n_songs_played", "n_unique_songs", "avg_song_length", "most_active_dow", "most_active_hour"]
for col in new_feature_cols:
    if col in test_features.columns:
        test_features[col] = test_features[col].fillna(0)

display(test_features.head())
print("test_features shape:", test_features.shape)


,userId,n_events,n_active_days,n_unique_pages,n_sessions,events_per_day,recency_days,account_age_days,last_level,n_help,n_upgrade,n_submit_downgrade,n_settings,n_about,n_error,n_songs_played,n_unique_songs,avg_song_length,most_active_dow,most_active_hour
0,1000655,346,11,11,10,31.454545,4,66,free,0.0,3.0,0.0,2.0,1.0,0.0,262.0,252.0,249.923186,6,22
1,1000963,2539,26,17,21,97.653846,1,73,paid,13.0,6.0,1.0,13.0,3.0,3.0,2112.0,1857.0,249.113424,3,17
2,1001129,668,10,15,8,66.800000,1,86,free,5.0,0.0,1.0,4.0,2.0,3.0,547.0,519.0,254.161281,4,2
3,1001963,718,16,14,16,44.875000,0,40,free,6.0,3.0,0.0,5.0,1.0,1.0,539.0,518.0,249.876958,4,14
4,1002283,3837,25,17,24,153.480000,0,54,paid,27.0,4.0,1.0,24.0,8.0,2.0,3160.0,2618.0,249.943611,2,18


test_features shape: (2904, 20)


In [49]:
# Appliquer les mêmes catégories que dans le train
test_features["last_level"] = pd.Categorical(
    test_features["last_level"],
    categories=last_level_cats
).codes

# Pour les NaN ou niveaux inconnus → -1, on peut les remplacer par 0
test_features["last_level"] = test_features["last_level"].replace(-1, 0)


In [50]:
# Renommer userId en id (si la colonne existe)
if "userId" in test_features.columns:
    test_features = test_features.rename(columns={"userId": "id"})
elif "id" not in test_features.columns:
    print("⚠️ Attention: ni 'userId' ni 'id' trouvé dans test_features")
    print("Colonnes disponibles:", list(test_features.columns))

# Harmoniser les types de id avant le merge
example["id"] = example["id"].astype(str)
if "id" in test_features.columns:
    test_features["id"] = test_features["id"].astype(str)
else:
    print("⚠️ Erreur: la colonne 'id' n'existe pas dans test_features")
    print("Colonnes disponibles:", list(test_features.columns))

# S'assurer que l'on garde uniquement les ids présents dans example, dans le même ordre
if "id" in test_features.columns:
    test_features = example[["id"]].merge(
        test_features,
        on="id",
        how="left"
    )
    print(f"✅ Merge réussi. Shape après merge: {test_features.shape}")
    print(f"Colonnes après merge: {list(test_features.columns)}")
else:
    print("⚠️ Impossible de faire le merge: colonne 'id' manquante")

display(test_features.head())
print("\nValeurs manquantes:")
print(test_features.isna().sum())


✅ Merge réussi. Shape après merge: (2904, 20)
Colonnes après merge: ['id', 'n_events', 'n_active_days', 'n_unique_pages', 'n_sessions', 'events_per_day', 'recency_days', 'account_age_days', 'last_level', 'n_help', 'n_upgrade', 'n_submit_downgrade', 'n_settings', 'n_about', 'n_error', 'n_songs_played', 'n_unique_songs', 'avg_song_length', 'most_active_dow', 'most_active_hour']


,id,n_events,n_active_days,n_unique_pages,n_sessions,events_per_day,recency_days,account_age_days,last_level,n_help,n_upgrade,n_submit_downgrade,n_settings,n_about,n_error,n_songs_played,n_unique_songs,avg_song_length,most_active_dow,most_active_hour
0,1128274,462,12,15,10,38.500000,5,78,0,3.0,8.0,1.0,3.0,0.0,1.0,359.0,348.0,245.626606,2,22
1,1782451,52,1,8,1,52.000000,45,53,0,0.0,0.0,0.0,0.0,2.0,0.0,39.0,39.0,247.594816,4,8
2,1611542,588,12,12,12,49.000000,3,107,0,2.0,0.0,0.0,7.0,1.0,1.0,479.0,463.0,248.012839,0,19
3,1241663,3079,28,16,25,109.964286,4,74,0,17.0,2.0,1.0,21.0,6.0,2.0,2561.0,2177.0,249.918453,4,22
4,1653104,1536,14,16,11,109.714286,1,84,0,5.0,1.0,0.0,8.0,1.0,1.0,1287.0,1182.0,246.740630,4,10



Valeurs manquantes:
id                    0
n_events              0
n_active_days         0
n_unique_pages        0
n_sessions            0
events_per_day        0
recency_days          0
account_age_days      0
last_level            0
n_help                0
n_upgrade             0
n_submit_downgrade    0
n_settings            0
n_about               0
n_error               0
n_songs_played        0
n_unique_songs        0
avg_song_length       0
most_active_dow       0
most_active_hour      0
dtype: int64


In [51]:
# Vérifier les colonnes disponibles
print("Colonnes dans test_features:", list(test_features.columns))

# On enlève id ET last_level pour garder exactement les features du modèle
# Vérifier que les colonnes existent avant de les supprimer
cols_to_drop = []
if "id" in test_features.columns:
    cols_to_drop.append("id")
if "last_level" in test_features.columns:
    cols_to_drop.append("last_level")

if cols_to_drop:
    X_test = test_features.drop(columns=cols_to_drop)
else:
    X_test = test_features.copy()

# On remplit les valeurs manquantes pour éviter les soucis
X_test = X_test.fillna(0)

print(f"\nX_test shape: {X_test.shape}")
print("Colonnes dans X_test:", list(X_test.columns))
X_test.head()



Colonnes dans test_features: ['id', 'n_events', 'n_active_days', 'n_unique_pages', 'n_sessions', 'events_per_day', 'recency_days', 'account_age_days', 'last_level', 'n_help', 'n_upgrade', 'n_submit_downgrade', 'n_settings', 'n_about', 'n_error', 'n_songs_played', 'n_unique_songs', 'avg_song_length', 'most_active_dow', 'most_active_hour']

X_test shape: (2904, 18)
Colonnes dans X_test: ['n_events', 'n_active_days', 'n_unique_pages', 'n_sessions', 'events_per_day', 'recency_days', 'account_age_days', 'n_help', 'n_upgrade', 'n_submit_downgrade', 'n_settings', 'n_about', 'n_error', 'n_songs_played', 'n_unique_songs', 'avg_song_length', 'most_active_dow', 'most_active_hour']


,n_events,n_active_days,n_unique_pages,n_sessions,events_per_day,recency_days,account_age_days,n_help,n_upgrade,n_submit_downgrade,n_settings,n_about,n_error,n_songs_played,n_unique_songs,avg_song_length,most_active_dow,most_active_hour
0,462,12,15,10,38.500000,5,78,3.0,8.0,1.0,3.0,0.0,1.0,359.0,348.0,245.626606,2,22
1,52,1,8,1,52.000000,45,53,0.0,0.0,0.0,0.0,2.0,0.0,39.0,39.0,247.594816,4,8
2,588,12,12,12,49.000000,3,107,2.0,0.0,0.0,7.0,1.0,1.0,479.0,463.0,248.012839,0,19
3,3079,28,16,25,109.964286,4,74,17.0,2.0,1.0,21.0,6.0,2.0,2561.0,2177.0,249.918453,4,22
4,1536,14,16,11,109.714286,1,84,5.0,1.0,0.0,8.0,1.0,1.0,1287.0,1182.0,246.740630,4,10


In [53]:
# Réordonner X_test pour avoir EXACTEMENT les mêmes colonnes que X_train
print(f"Colonnes attendues par le modèle ({len(cols_order)}): {list(cols_order)}")
print(f"Colonnes dans X_test ({len(X_test.columns)}): {list(X_test.columns)}")

# Vérifier les colonnes manquantes
missing_cols = [c for c in cols_order if c not in X_test.columns]
if missing_cols:
    print(f"⚠️ Colonnes manquantes: {missing_cols}")
    # Créer les colonnes manquantes avec des 0
    for col in missing_cols:
        X_test[col] = 0

# Réordonner selon l'ordre du train
X_test = X_test[cols_order]

print(f"\n✅ Colonnes finales de X_test ({len(X_test.columns)}): {list(X_test.columns)}")
print(f"Shape X_test: {X_test.shape}")


Colonnes attendues par le modèle (18): ['n_events', 'n_active_days', 'n_unique_pages', 'n_sessions', 'events_per_day', 'recency_days', 'account_age_days', 'n_help', 'n_upgrade', 'n_submit_downgrade', 'n_settings', 'n_about', 'n_error', 'n_songs_played', 'n_unique_songs', 'avg_song_length', 'most_active_dow', 'most_active_hour']
Colonnes dans X_test (18): ['n_events', 'n_active_days', 'n_unique_pages', 'n_sessions', 'events_per_day', 'recency_days', 'account_age_days', 'n_help', 'n_upgrade', 'n_submit_downgrade', 'n_settings', 'n_about', 'n_error', 'n_songs_played', 'n_unique_songs', 'avg_song_length', 'most_active_dow', 'most_active_hour']

✅ Colonnes finales de X_test (18): ['n_events', 'n_active_days', 'n_unique_pages', 'n_sessions', 'events_per_day', 'recency_days', 'account_age_days', 'n_help', 'n_upgrade', 'n_submit_downgrade', 'n_settings', 'n_about', 'n_error', 'n_songs_played', 'n_unique_songs', 'avg_song_length', 'most_active_dow', 'most_active_hour']
Shape X_test: (2904, 18)


In [ ]:
test_proba = xgb.predict_proba(X_test)[:, 1]

print("Min / max des proba sur test :")
print(test_proba.min(), test_proba.max())





Min / max des proba sur test :
0.0055507747 0.8693498


In [ ]:
# Analyser la distribution des probabilités
import numpy as np

print("📊 Distribution des probabilités sur test :")
print(f"Min: {test_proba.min():.4f}")
print(f"Max: {test_proba.max():.4f}")
print(f"Moyenne: {test_proba.mean():.4f}")
print(f"Médiane: {np.median(test_proba):.4f}")
print(f"\nPercentiles:")
for p in [50, 75, 90, 95, 99]:
    print(f"  {p}ème percentile: {np.percentile(test_proba, p):.4f}")

# Le taux de churn réel dans le train est ~4.3% (663/15276)
# On devrait avoir un taux de prédictions positives similaire
churn_rate_train = 663 / 15276
print(f"\n📈 Taux de churn dans le train: {churn_rate_train:.2%}")

# Trouver un seuil qui donne un taux de prédictions positives proche du taux réel
# Option 1: Utiliser le percentile correspondant au taux de churn
target_percentile = (1 - churn_rate_train) * 100
threshold_percentile = np.percentile(test_proba, target_percentile)
print(f"\nSeuil basé sur le percentile {target_percentile:.1f}%: {threshold_percentile:.4f}")

# Option 2: Tester plusieurs seuils et voir la distribution
print("\n📋 Test de différents seuils:")
for thresh in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]:
    preds = (test_proba >= thresh).astype(int)
    rate = preds.mean()
    print(f"  Seuil {thresh:.1f}: {preds.sum()} prédictions positives ({rate:.1%})")

# Pour l'instant, utilisons un seuil plus conservateur
# Avec un max de 0.82, un seuil autour de 0.3-0.4 semble plus raisonnable
# que 0.49 qui donne trop de prédictions positives
threshold = 0.56  # À ajuster selon les résultats

test_pred = (test_proba >= threshold).astype(int)

print(f"\n✅ Seuil choisi: {threshold}")
print("\nRépartition finale des prédictions (0/1) :")
print(pd.Series(test_pred).value_counts().sort_index())
print(f"Taux de prédictions positives: {test_pred.mean():.2%}")


📊 Distribution des probabilités sur test :
Min: 0.0056
Max: 0.8693
Moyenne: 0.4091
Médiane: 0.4279

Percentiles:
  50ème percentile: 0.4279
  75ème percentile: 0.5778
  90ème percentile: 0.6604
  95ème percentile: 0.7013
  99ème percentile: 0.7812

📈 Taux de churn dans le train: 4.34%

Seuil basé sur le percentile 95.7%: 0.7095

📋 Test de différents seuils:
  Seuil 0.1: 2667 prédictions positives (91.8%)
  Seuil 0.2: 2299 prédictions positives (79.2%)
  Seuil 0.3: 1956 prédictions positives (67.4%)
  Seuil 0.4: 1576 prédictions positives (54.3%)
  Seuil 0.5: 1152 prédictions positives (39.7%)
  Seuil 0.6: 610 prédictions positives (21.0%)

✅ Seuil choisi: 0.56

Répartition finale des prédictions (0/1) :
0    2063
1     841
Name: count, dtype: int64
Taux de prédictions positives: 28.96%


In [57]:
# Générer la soumission avec le seuil choisi
submission = pd.DataFrame({
    "id": test_features["id"],
    "target": test_pred  # labels 0/1 pour Kaggle
})

print(submission.dtypes)
display(submission.head())

submission.to_csv("submission_time_based.csv", index=False)
print("✅ submission_time_based.csv (labels 0/1) generated!")
print(f"   Taux de prédictions positives: {test_pred.mean():.2%}")
print(f"   Nombre de prédictions positives: {test_pred.sum()}")


id        object
target     int64
dtype: object


,id,target
0,1128274,0
1,1782451,0
2,1611542,0
3,1241663,0
4,1653104,1


✅ submission_time_based.csv (labels 0/1) generated!
   Taux de prédictions positives: 28.96%
   Nombre de prédictions positives: 841


In [ ]:
# Tester plusieurs seuils et générer plusieurs soumissions pour trouver le meilleur
# Le taux de churn réel est ~4.34%, donc on devrait viser un taux de prédictions positives similaire

print("🔍 Génération de plusieurs soumissions avec différents seuils...\n")

# Seuils à tester basés sur les percentiles et le taux de churn réel
thresholds_to_test = [
    0.30,  # Plus permissif
    0.35,  # 
    0.40,  # 
    0.44,  # Optimal F1-score (actuel)
    0.50,  # 
    0.55,  # 
    0.60,  # 
    0.64,  # Percentile 95.7% (taux churn réel)
    0.70,  # Plus conservateur
]

for thresh in thresholds_to_test:
    preds = (test_proba >= thresh).astype(int)
    rate = preds.mean()
    n_pos = preds.sum()
    
    submission_test = pd.DataFrame({
        "id": test_features["id"],
        "target": preds
    })
    
    filename = f"submission_thresh_{thresh:.2f}.csv"
    submission_test.to_csv(filename, index=False)
    
    print(f"✅ {filename}")
    print(f"   Seuil: {thresh:.2f} | Positives: {n_pos} ({rate:.2%})")
    print()

print("💡 Suggestions:")
print("   - Si Kaggle utilise AUC: le seuil n'a pas d'importance, le problème vient du modèle")
print("   - Si Kaggle utilise F1/Accuracy: tester les seuils autour de 0.44 (optimal F1)")
print("   - Pour un taux proche du churn réel (4.34%): utiliser ~0.64")
print("   - Score actuel: 0.56336 - très proche de 0.5 (aléatoire)")
print("   - Le modèle pourrait nécessiter des améliorations (features, hyperparamètres)")


🔍 Génération de plusieurs soumissions avec différents seuils...

✅ submission_thresh_0.30.csv
   Seuil: 0.30 | Positives: 1151 (39.63%)

✅ submission_thresh_0.35.csv
   Seuil: 0.35 | Positives: 954 (32.85%)

✅ submission_thresh_0.40.csv
   Seuil: 0.40 | Positives: 778 (26.79%)

✅ submission_thresh_0.44.csv
   Seuil: 0.44 | Positives: 645 (22.21%)

✅ submission_thresh_0.50.csv
   Seuil: 0.50 | Positives: 455 (15.67%)

✅ submission_thresh_0.55.csv
   Seuil: 0.55 | Positives: 316 (10.88%)

✅ submission_thresh_0.60.csv
   Seuil: 0.60 | Positives: 197 (6.78%)

✅ submission_thresh_0.64.csv
   Seuil: 0.64 | Positives: 127 (4.37%)

✅ submission_thresh_0.70.csv
   Seuil: 0.70 | Positives: 63 (2.17%)

💡 Suggestions:
   - Si Kaggle utilise AUC: le seuil n'a pas d'importance, le problème vient du modèle
   - Si Kaggle utilise F1/Accuracy: tester les seuils autour de 0.44 (optimal F1)
   - Pour un taux proche du churn réel (4.34%): utiliser ~0.64
   - Score actuel: 0.56336 - très proche de 0.5 (alé